In [11]:
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt

from time import time
from functools import partial

import sys
sys.path.append('../../')

from Environment import Quadrotor1D
from Subroutines import QuadrotorEst,ApproxDAP,max_norm,find_stable_radius,SafeTransit,spectral_radius
from time import time
from Controllers import SafeDAP
from scipy.linalg import sqrtm
import pickle as pkl
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
class TimeStepsReached(Exception):
    pass
def evaluate(space_dim,dt,K_m_dt,l_m_dt):
    O = np.zeros((space_dim,space_dim))
    I = np.eye(space_dim)

    A = np.vstack([np.hstack([O,dt*I]),
                   np.hstack([-K_m_dt*I,-l_m_dt*I])]) + np.eye(2*space_dim)

    return  np.max(np.abs(np.linalg.eigvals(A)))

# Set up the system

In [13]:
m = 1
K_stab = np.array([[2/3,1]])
l = 0.25
dt = 1
w_max = 0.2

env=Quadrotor1D(m,K_stab,l,w_max,dt)

# $\Theta_{ini}$

In [14]:
alpha_limit = (0.5,1.2)
beta_limit = (0.2,0.4)

# Optimization Parameters

In [15]:
# Constants
x_dim = env.AK.shape[-1]
u_dim = env.B.shape[-1]


I_x = np.eye(x_dim)  
I_u = np.eye(u_dim)  

w_cov = np.eye(x_dim) * env.w_max * 1/12 # Assume uniform distribution.

# Cost matrices

Q = np.array([[1,0],
                [0,1]])*1
R = np.eye(u_dim)*1

# Constraints

x_max = 5
x_min = -1

v_max = 2
v_min = -2

u_max = 5
u_min = -9.8

D_x = np.vstack([I_x,-I_x]) # Constraints
d_x = np.array([x_max,v_max,-x_min,-v_min])

D_u = np.vstack([I_u,-I_u])
d_u = np.array([u_max,-u_min])

# Lookback lengths
H=10
H_benchmark = 40

M0 = np.zeros((H,u_dim,x_dim))

e_x = 0 # Usually e_x is not zero 
e_u = 0 # For safe DAP with known B, e_u = 0

refit_per_step = 50
pre_run_steps = 2
TD_steps  = 30 # The least number of steps taken in Phase 1.

# eta_bars = [0.1,0.5,1.0,1.5,2.0]
eta_bars = [0.1]

# Save a copy of the parameters we have

In [16]:
params = {'Q':Q,'R':R,'D_x':D_x,'d_x':d_x,'D_u':D_u,'d_u':d_u,\
          'alpha_limit':alpha_limit,'beta_limit':beta_limit,\
          'H':H,'H_benchmark':H_benchmark,'eta_bars':eta_bars,\
          'AK':env.AK,'B':env.B,\
          'w_max':w_max,'K_stab':K_stab,'l':l,'m':m,'dt':dt}
with open('./data/Parameters.pkl','wb') as f:
    pkl.dump(params,f)

# Experiments

In [7]:

def PlainSim(timesteps,n_trials):
    
    def main_loop():
        env=Quadrotor1D(m,K_stab,l,w_max,dt)
        
        x_no_control_hist = []
        # main loop
        for _ in range(timesteps):
            x = env.state()
            u = 0 
            env.step(u)

            x_no_control_hist.append(x)

            if _%100 == 0:
                print('Step',_)

        data = {}
        data['x'] = x_no_control_hist
        return data
    
    trial_data = []
    _ = 0 
    while _ < n_trials: 
        t = time()
        print('Trial {}'.format(_))
        trial_data.append(main_loop())
        _+=1

        print('Time for trial:',time()-t)
    return trial_data

def BenchmarkSim(timesteps,n_trials,unconstrained=False,with_K_stab = False):

    def main_loop(M):
        # Reset the environment
        env=Quadrotor1D(m,K_stab,l,w_max,dt)
        
        x_true_hist = []
        u_true_hist = []
        w_true_hist = [np.zeros((x_dim,1)) for _ in range(10*H_benchmark)]

        # main loop 
        for _ in range(timesteps):
            x = env.state()
            x_true_hist.append(x)
            if _<pre_run_steps:
                u = (np.random.rand()-0.5)*2*0
            else:
                u = ApproxDAP(M,w_true_hist,0)

            env.step(u)
            u_true_hist.append(u)

            w_true_hist.append(env.state()-env.AK.dot(x_true_hist[-1])-env.B.dot(u_true_hist[-1]))

            if _%100 == 0:
                print('Step',_)

        # Prepare data output
        data = {}
        data['x'] = x_true_hist
        data['u'] = u_true_hist
        data['w'] = w_true_hist
        
#         print(data['w'])
        return data



    safeDapSolver = SafeDAP(Q,R,D_x,d_x,D_u,d_u,w_max,w_cov)

    if with_K_stab:    
        if unconstrained:
            M,Phi = safeDapSolver.solve(env.AK,env.B,H_benchmark,e_x=e_x,e_u=e_u,unconstrained=True,K_stab=K_stab)
        else:
            M,Phi = safeDapSolver.solve(env.AK,env.B,H_benchmark,e_x=e_x,e_u=e_u,K_stab=K_stab)
    else:        
        if unconstrained:
            M,Phi = safeDapSolver.solve(env.AK,env.B,H_benchmark,e_x=e_x,e_u=e_u,unconstrained=True)
        else:
            M,Phi = safeDapSolver.solve(env.AK,env.B,H_benchmark,e_x=e_x,e_u=e_u)

        
#     print('M',M)

    trial_data = []
    _ = 0 
    while _ < n_trials: 

        print('Trial {}'.format(_))
        trial_data.append(main_loop(M))
        _+=1
        
    return trial_data    

In [8]:

n_trials = 3
timesteps = 400

alg_dict = {
    'ConstrainedDAP_with_Kstab':partial(BenchmarkSim,unconstrained=False,with_K_stab=True),
    'UnconstrainedDAP':partial(BenchmarkSim,unconstrained=True,with_K_stab=True),
              }


for key, alg in alg_dict.items():
    print("{} starts.".format(key))
    
    t = time()
      

    trial_data = alg(timesteps,n_trials)
    with open('./data/{}.pkl'.format(key),'wb') as f:
        pkl.dump(trial_data,f)
    
    print('Total Time for {}:'.format(key),time()-t)


ConstrainedDAP_with_Kstab starts.
Trial 0
Step 0
Step 100
Step 200
Step 300
Trial 1
Step 0
Step 100
Step 200
Step 300
Trial 2
Step 0
Step 100
Step 200
Step 300
Total Time for ConstrainedDAP_with_Kstab: 2.32442307472229
ConstrainedDAP_without_Kstab starts.
Trial 0
Step 0
Step 100
Step 200
Step 300
Trial 1
Step 0
Step 100
Step 200
Step 300
Trial 2
Step 0
Step 100
Step 200
Step 300
Total Time for ConstrainedDAP_without_Kstab: 2.240206480026245
UnconstrainedDAP_with_Kstab starts.
Trial 0
Step 0
Step 100
Step 200
Step 300
Trial 1
Step 0
Step 100
Step 200
Step 300
Trial 2
Step 0
Step 100
Step 200
Step 300
Total Time for UnconstrainedDAP_with_Kstab: 0.9362409114837646
UnconstrainedDAP_without_Kstab starts.
Trial 0
Step 0
Step 100
Step 200
Step 300
Trial 1
Step 0
Step 100
Step 200
Step 300
Trial 2
Step 0
Step 100
Step 200
Step 300
Total Time for UnconstrainedDAP_without_Kstab: 0.9232039451599121


In [9]:
est = QuadrotorEst(K_stab,dt,alpha_limit,beta_limit)

In [10]:
est.est(np.array(trial_data[0]['x']).reshape(-1,x_dim),np.array(trial_data[0]['u'][:-1]).reshape(-1,u_dim))

(array([[1. , 1. ],
        [0. , 0.6]]),
 array([[0. ],
        [0.5]]),
 0.12262786789699316)